In [27]:
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *
from pandas import merge, read_excel, pivot_table, melt, to_datetime, to_timedelta, DateOffset, Timestamp, Period, Series
import numpy as np
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
origin = my_read_excel("통합5-3.xlsx")
origin['연월'] = to_datetime(origin['연월'])
df = origin.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10028 entries, 0 to 10027
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   회원카드번호   10028 non-null  int64 
 1   연월       10028 non-null  object
 2   요일       10028 non-null  object
 3   미수액      10028 non-null  int64 
 4   FC 강사    9293 non-null   object
 5   등록명      6497 non-null   object
 6   일일입장     10028 non-null  int64 
 7   분실       10028 non-null  int64 
 8   부가서비스사용  10028 non-null  int64 
 9   레슨여부     10028 non-null  int64 
 10  매출액      10028 non-null  int64 
 11  개월수      10028 non-null  int64 
 12  골프전환     10028 non-null  int64 
 13  단체회원     10028 non-null  int64 
 14  프로모션     10028 non-null  object
 15  단체명      2497 non-null   object
 16  패스       10028 non-null  object
 17  환불내역     10028 non-null  object
 18  거래일_일    10028 non-null  int64 
 19  환불일_일    10028 non-null  int64 
 20  골프매출     10028 non-null  int64 
 21  라카매출     10028 non-null  int64 
 22

In [29]:
df1 = df.filter(['회원카드번호', '연월','등록명', '매출액','골프전환','골프매출','헬스매출','환불사용금매출','미수액', '개월수'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10028 entries, 0 to 10027
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   회원카드번호   10028 non-null  int64         
 1   연월       10028 non-null  datetime64[ns]
 2   등록명      6497 non-null   object        
 3   매출액      10028 non-null  int64         
 4   골프전환     10028 non-null  int64         
 5   골프매출     10028 non-null  int64         
 6   헬스매출     10028 non-null  int64         
 7   환불사용금매출  10028 non-null  int64         
 8   미수액      10028 non-null  int64         
 9   개월수      10028 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 783.6+ KB


In [30]:
for i in range(len(df1)):
    # Convert Period to Timestamp, add DateOffset, and then convert back to Period
    df1.loc[i, '종료시점'] = (df1.loc[i, '연월']+ 
                             DateOffset(months=df1.loc[i, '개월수']))
df1

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점
0,12131325,2020-09-01,헬스,899000,0,0,899000,0,0,12,2021-09-01
1,13110008,2021-05-01,골프,1700000,0,1580000,0,0,0,12,2022-05-01
2,13110008,2021-06-01,NaN,200000,0,0,0,0,0,0,2021-06-01
3,13110008,2022-06-01,골프,1700000,0,1580000,0,0,0,12,2023-06-01
4,13110008,2023-08-01,NaN,220000,0,0,0,0,0,0,2023-08-01
...,...,...,...,...,...,...,...,...,...,...,...
10023,23120077,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01
10024,24010001,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01
10025,24010002,2023-12-01,NaN,990000,0,0,0,0,0,0,2023-12-01
10026,24010003,2022-01-01,헬스,1035000,0,0,990000,0,0,10,2022-11-01


In [31]:
df1.info()
# Convert '종료시점' column to Period with frequency 'M'
# df1['종료시점'] = df1['종료시점'].dt.to_period("M")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10028 entries, 0 to 10027
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   회원카드번호   10028 non-null  int64         
 1   연월       10028 non-null  datetime64[ns]
 2   등록명      6497 non-null   object        
 3   매출액      10028 non-null  int64         
 4   골프전환     10028 non-null  int64         
 5   골프매출     10028 non-null  int64         
 6   헬스매출     10028 non-null  int64         
 7   환불사용금매출  10028 non-null  int64         
 8   미수액      10028 non-null  int64         
 9   개월수      10028 non-null  int64         
 10  종료시점     10028 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(8), object(1)
memory usage: 861.9+ KB


#### 회원별 최초등록일

In [32]:
df2 = df1[['회원카드번호', '연월']].groupby(by= '회원카드번호', as_index =False).min()
df2.rename(columns={'연월':'최초등록월'}, inplace=True)
df3 = merge(df1, df2, on =['회원카드번호'])
df3

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월
0,12131325,2020-09-01,헬스,899000,0,0,899000,0,0,12,2021-09-01,2020-09-01
1,13110008,2021-05-01,골프,1700000,0,1580000,0,0,0,12,2022-05-01,2021-05-01
2,13110008,2021-06-01,NaN,200000,0,0,0,0,0,0,2021-06-01,2021-05-01
3,13110008,2022-06-01,골프,1700000,0,1580000,0,0,0,12,2023-06-01,2021-05-01
4,13110008,2023-08-01,NaN,220000,0,0,0,0,0,0,2023-08-01,2021-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...
10023,23120077,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01
10024,24010001,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01
10025,24010002,2023-12-01,NaN,990000,0,0,0,0,0,0,2023-12-01,2023-12-01
10026,24010003,2022-01-01,헬스,1035000,0,0,990000,0,0,10,2022-11-01,2022-01-01


In [81]:
# 제외할거래 = df3[df3['종료시점']>= to_datetime("2024-01", format="%Y-%m")]
# 제외할거래
# df4 = df3[~df3.index.isin(제외할거래.index)] #날짜 컬럼이 포함된 행은 ~ 연산자를 적용할 수 없음.

In [33]:
df4 = df3.copy()
df4[(df4['개월수'] ==0)]['등록명'].unique()

array([nan, '헬스', '골프', '골프골프', '헬스헬스', '골프헬스', '헬스골프', '골프골프헬스',
       '헬스헬스골프'], dtype=object)

In [34]:
등록명 = ['헬스', '골프', '골프골프', '헬스헬스', '골프헬스', '헬스골프', '골프골프헬스',
       '헬스헬스골프']
df4[(df4['개월수'] == 0)&(~df4['등록명'].isin(등록명))]

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월
2,13110008,2021-06-01,NaN,200000,0,0,0,0,0,0,2021-06-01,2021-05-01
4,13110008,2023-08-01,NaN,220000,0,0,0,0,0,0,2023-08-01,2021-05-01
12,13110017,2021-01-01,NaN,180000,0,0,0,0,0,0,2021-01-01,2020-06-01
16,13110019,2020-05-01,NaN,495000,0,0,0,0,0,0,2020-05-01,2020-03-01
19,13110030,2020-03-01,NaN,30000,0,0,0,0,0,0,2020-03-01,2020-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...
9938,23110048,2023-11-01,NaN,990000,0,0,0,0,0,0,2023-11-01,2023-11-01
9944,23110052,2023-12-01,NaN,120000,0,0,0,0,0,0,2023-12-01,2023-11-01
9946,23110053,2023-12-01,NaN,45000,0,0,0,0,0,0,2023-12-01,2023-11-01
10010,23120064,2023-12-01,NaN,880000,0,0,0,0,0,0,2023-12-01,2023-12-01


In [36]:
import numpy as np

# Assuming your array is named 'values'
nan_indices = np.isnan(df4['개월수'])
clean_values = df4['개월수'][~nan_indices]
df4[nan_indices]
#df4.head()


,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월


### 개월수만 고려 (등록명이 없는 항목은 매출이 있어도 제외함.)

In [37]:
df4['등록명'].unique()

array(['헬스', '골프', nan, '골프골프', '헬스헬스', '골프헬스', '헬스헬스헬스', '헬스골프',
       '골프골프헬스', '골프골프골프', '골프골프골프골프', '헬스헬스골프', '골프골프골프헬스', '골프헬스골프'],
      dtype=object)

In [38]:

df4['등록명'].unique()
등록명 = ['헬스', '골프', '골프골프', '헬스헬스', '골프헬스', '헬스헬스헬스', '헬스골프',
       '골프골프헬스', '골프골프골프', '골프골프골프골프', '헬스헬스골프', '골프골프골프헬스', '골프헬스골프']
df5 = df4[df4['등록명'].isin(등록명)]
df5.reset_index(drop=True, inplace=True)

In [87]:
# 행병합 = []

# a = df5.query('회원카드번호 == 13110008.0').sort_values(by="연월")
# a.reset_index(drop=True, inplace=True)
# for j in range(0, len(a)): # 0, 1
#         a.loc[j, '거래순서'] = j  # 거래순서 열 추가 # pandas ranking 써 볼

# 행병합.append(a)  # 행병합 리스트에 데이터프레임 추가

# 행병합
# df6= concat(행병합, ignore_index=True)
# df6

In [39]:
from datetime import datetime

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# diff_month(datetime.today(), datetime(2020, 1, 20))

### 환불회원 개월수 보정

In [40]:
회원카드 = df5['회원카드번호'].unique()
행병합 = []

for i in 회원카드:
    a = df5.query('회원카드번호 == @i').sort_values(by="연월")  # 연월에 대해 데이터프레임을 정렬
    a.reset_index(drop=True, inplace=True)

    for j in range(len(a)):
        if j > 0:
            if (a.loc[j, '골프매출'] < 0) and (a.loc[j, '개월수'] < 0):
                if -a.loc[j,'골프매출'] == a.loc[j-1,'골프매출']:
                    a.loc[j, "개월수"] -= diff_month(a.loc[j-1, '연월'], a.loc[j,'연월'])
                    a.loc[j, "환불여부"] = 1
            if (a.loc[j, '헬스매출'] < 0) and (a.loc[j, '개월수'] < 0):
                if -a.loc[j,'헬스매출'] == a.loc[j-1,'헬스매출']:
                    a.loc[j, "개월수"] -= diff_month(a.loc[j-1, '연월'], a.loc[j,'연월'])
                    a.loc[j, "환불여부"] = 1

    행병합.append(a)

df6 = concat(행병합, ignore_index=True)
df9 = df6[df6["환불여부"]==1]
for i in df9.index:
    df6.loc[i, '개월수'] = df9.loc[i, '개월수']
    df6.loc[i, '환불여부'] = df9.loc[i, '환불여부']

# diff_month(df5.loc[df5['회원카드번호'==13110011], '연월'], a.loc[j,'연월'])

### 골프전환 개월수 처리

In [41]:

def custom_date_parser(date_string):
    return to_datetime(date_string, format="%Y-%m-%d %H:%M:%S")

golforigin = read_excel('골프전환개월수_매칭용.xlsx', parse_dates=['거래일시'], date_parser=custom_date_parser)
golfdf = golforigin.copy()
# golfdf['연월'] = to_datetime(golfdf['거래일시']).dt.to_period("M")
golfdf['연월'] = golfdf['거래일시'].dt.strftime('%Y-%m')
golfdf['연월'] = to_datetime(golfdf['연월'], format='%Y-%m')

golfdf2 = golfdf[['회원카드번호', '연월','골프전환개월수']].groupby(['회원카드번호', '연월'], as_index=False).sum()
df_golfmonth = merge(df6, golfdf2,left_on=['회원카드번호', '연월'], right_on=['회원카드번호', '연월'], how='left')


In [42]:
for i in df_golfmonth.index:
    if df_golfmonth.loc[i, '골프전환개월수'] == df_golfmonth.loc[i, '골프전환개월수']:
        df_golfmonth.loc[i, '개월수'] = df_golfmonth.loc[i, '골프전환개월수']
        df_golfmonth.loc[i, '골프전환2'] = 1
df5 = df_golfmonth.drop(['골프전환개월수'], axis=1)
df5

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월,환불여부,골프전환2
0,12131325,2020-09-01,헬스,899000,0,0,899000,0,0,12,2021-09-01,2020-09-01,NaN,NaN
1,13110008,2021-05-01,골프,1700000,0,1580000,0,0,0,12,2022-05-01,2021-05-01,NaN,NaN
2,13110008,2022-06-01,골프,1700000,0,1580000,0,0,0,12,2023-06-01,2021-05-01,NaN,NaN
3,13110008,2023-11-01,골프,2120000,0,1880000,0,0,0,12,2024-11-01,2021-05-01,NaN,NaN
4,13110009,2020-10-01,헬스,1140000,0,0,1080000,0,0,12,2021-10-01,2020-10-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,23120076,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN
6493,23120077,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN
6494,24010001,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN
6495,24010003,2022-01-01,헬스,1035000,0,0,990000,0,0,10,2022-11-01,2022-01-01,NaN,NaN


In [43]:
df5[df5['회원카드번호']==13110030]

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월,환불여부,골프전환2
15,13110030,2021-02-01,골프,2090000,0,1480000,0,0,0,12,2022-02-01,2020-03-01,NaN,NaN
16,13110030,2022-05-01,헬스,0,0,0,0,0,0,0,2022-05-01,2020-03-01,NaN,NaN
17,13110030,2022-08-01,골프,290000,1,90000,0,0,0,0,2022-08-01,2020-03-01,NaN,NaN
18,13110030,2022-11-01,헬스,3120000,0,0,1080000,0,0,12,2023-11-01,2020-03-01,NaN,NaN


In [44]:
df5['등록명'].unique()

array(['헬스', '골프', '골프골프', '헬스헬스', '골프헬스', '헬스헬스헬스', '헬스골프', '골프골프헬스',
       '골프골프골프', '골프골프골프골프', '헬스헬스골프', '골프골프골프헬스', '골프헬스골프'], dtype=object)

In [45]:
df5[df5['등록명']=='골프헬스헬스']

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월,환불여부,골프전환2


In [46]:
df5[(df5['골프전환']==1)&(df5['개월수']==0)]

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월,환불여부,골프전환2
17,13110030,2022-08-01,골프,290000,1,90000,0,0,0,0,2022-08-01,2020-03-01,NaN,NaN
34,13110069,2021-08-01,골프,690000,1,590000,0,0,0,0,2021-08-01,2020-05-01,NaN,NaN
80,13120014,2021-07-01,골프골프,880000,1,390000,0,0,0,0,2021-07-01,2020-04-01,NaN,NaN
185,13120178,2022-05-01,골프,360000,1,240000,0,0,0,0,2022-05-01,2020-02-01,NaN,NaN
198,13120195,2021-02-01,골프,150000,1,150000,0,0,0,0,2021-02-01,2020-02-01,NaN,NaN
288,13120342,2022-03-01,골프,360000,1,360000,0,0,0,0,2022-03-01,2020-01-01,NaN,NaN
473,14020127,2023-08-01,골프,230000,1,230000,0,0,0,0,2023-08-01,2020-05-01,NaN,NaN
488,14020148,2021-12-01,골프,520000,1,0,0,0,0,0,2021-12-01,2020-03-01,NaN,NaN
496,14020156,2022-10-01,골프,0,1,0,0,0,0,0,2022-10-01,2020-02-01,NaN,NaN
567,14040103,2021-03-01,골프,240000,1,240000,0,0,0,0,2021-03-01,2020-01-01,NaN,NaN


### 양도개월수 적용
- 양도한 사람은 기존 개월수 + (- 양도개월수)
- 양도 받은 사람은 기존 개월수를 양도개월수로 변경

In [47]:
yang_origin = read_excel('번호별연월별양도리스트.xlsx', )
ydf = yang_origin.copy()[['회원카드번호', '연월', '양도개월수']]
ydf['연월'] = to_datetime(ydf['연월'], format='%Y-%m')
df_yang_month = merge(df5, ydf,left_on=['회원카드번호', '연월'], right_on=['회원카드번호', '연월'], how='left')


In [48]:
for i in df_yang_month.index:
    if df_yang_month.loc[i, '양도개월수'] == df_yang_month.loc[i, '양도개월수']:
        if df_yang_month.loc[i, '양도개월수'] < 0:
            df_yang_month.loc[i, '개월수'] =  df_yang_month.loc[i, '개월수'] + df_yang_month.loc[i, '양도개월수']
            df_yang_month.loc[i, '양도'] = 1
        elif df_yang_month.loc[i, '양도개월수'] >= 0:
            df_yang_month.loc[i, '개월수'] =  df_yang_month.loc[i, '양도개월수']
            df_yang_month.loc[i, '양도'] = 1

df5= df_yang_month.drop(['양도개월수'], axis=1)
df5

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월,환불여부,골프전환2,양도
0,12131325,2020-09-01,헬스,899000,0,0,899000,0,0,12,2021-09-01,2020-09-01,NaN,NaN,NaN
1,13110008,2021-05-01,골프,1700000,0,1580000,0,0,0,12,2022-05-01,2021-05-01,NaN,NaN,NaN
2,13110008,2022-06-01,골프,1700000,0,1580000,0,0,0,12,2023-06-01,2021-05-01,NaN,NaN,NaN
3,13110008,2023-11-01,골프,2120000,0,1880000,0,0,0,12,2024-11-01,2021-05-01,NaN,NaN,NaN
4,13110009,2020-10-01,헬스,1140000,0,0,1080000,0,0,12,2021-10-01,2020-10-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,23120076,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN,NaN
6493,23120077,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN,NaN
6494,24010001,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN,NaN
6495,24010003,2022-01-01,헬스,1035000,0,0,990000,0,0,10,2022-11-01,2022-01-01,NaN,NaN,NaN


### 직원카드번호 제외

In [49]:
직원카드번호 = my_read_excel("직원카드번호.xlsx")
직원 = 직원카드번호['회원카드번호'].unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회원카드번호  98 non-null     int64 
 1   직원이름    98 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
None

상위 5개 행
+----+----------------+------------------+
|    |   회원카드번호 | 직원이름         |
|----+----------------+------------------|
|  0 |       20060022 | 홍진석(직원)     |
|  1 |       21110025 | 손재욱(직원가족) |
|  2 |       21110025 | 손재욱(직원가족) |
|  3 |       21110025 | 손재욱(직원가족) |
|  4 |       23050073 | 윤성현cs(직원)   |
+----+----------------+------------------+

하위 5개 행
+----+----------------+----------------+
|    |   회원카드번호 | 직원이름       |
|----+----------------+----------------|
| 93 |       23050073 | 윤성현cs(직원) |
| 94 |       23020053 | 백희정(직원)   |
| 95 |       23050075 | 김광현(직원)   |
| 96 |       23020052 | 신지연(직원)   |
| 97 |       23050074 | 홍석민(직원)   |
+----+----------------+----------------+



In [50]:
df5=df5[~df5['회원카드번호'].isin(직원)]
df5.reset_index(drop=True, inplace=True)
df5

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,종료시점,최초등록월,환불여부,골프전환2,양도
0,12131325,2020-09-01,헬스,899000,0,0,899000,0,0,12,2021-09-01,2020-09-01,NaN,NaN,NaN
1,13110008,2021-05-01,골프,1700000,0,1580000,0,0,0,12,2022-05-01,2021-05-01,NaN,NaN,NaN
2,13110008,2022-06-01,골프,1700000,0,1580000,0,0,0,12,2023-06-01,2021-05-01,NaN,NaN,NaN
3,13110008,2023-11-01,골프,2120000,0,1880000,0,0,0,12,2024-11-01,2021-05-01,NaN,NaN,NaN
4,13110009,2020-10-01,헬스,1140000,0,0,1080000,0,0,12,2021-10-01,2020-10-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,23120076,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN,NaN
6444,23120077,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN,NaN
6445,24010001,2023-12-01,헬스,990000,0,0,990000,0,0,10,2024-10-01,2023-12-01,NaN,NaN,NaN
6446,24010003,2022-01-01,헬스,1035000,0,0,990000,0,0,10,2022-11-01,2022-01-01,NaN,NaN,NaN


### 매출액, 골프전환, 골프매출, 헬스매출, 환불사용금매출, 미수액, 개월수가 모두 0인 등록명= '헬스'이거나 '골프'인 항목 제거

In [51]:
df100 = df5[(df5['매출액']==0)&(df5['골프전환']==0)&(df5['골프매출']==0)&(df5['헬스매출']==0)&(df5['환불사용금매출']==0)&(df5['미수액']==0)&(df5['개월수']==0)&(df5['등록명']=='헬스')]
df_filtered = df5[~df5.index.isin(df100.index)]
df200 = df5[(df5['매출액']==0)&(df5['골프전환']==0)&(df5['골프매출']==0)&(df5['헬스매출']==0)&(df5['환불사용금매출']==0)&(df5['미수액']==0)&(df5['개월수']==0)&(df5['등록명']=='골프')]
df_filtered2 = df_filtered[~df_filtered.index.isin(df200.index)] 
df5 = df_filtered2.copy()
df5.reset_index(drop=True, inplace=True)

In [52]:
df5[df5['개월수']==0]['등록명'].unique()

array(['골프', '골프골프', '헬스', '골프헬스', '헬스헬스', '골프골프헬스', '헬스골프'], dtype=object)

In [102]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6311 entries, 0 to 6310
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   회원카드번호   6311 non-null   int64         
 1   연월       6311 non-null   datetime64[ns]
 2   등록명      6311 non-null   object        
 3   매출액      6311 non-null   int64         
 4   골프전환     6311 non-null   int64         
 5   골프매출     6311 non-null   int64         
 6   헬스매출     6311 non-null   int64         
 7   환불사용금매출  6311 non-null   int64         
 8   미수액      6311 non-null   int64         
 9   개월수      6311 non-null   int64         
 10  종료시점     6311 non-null   datetime64[ns]
 11  최초등록월    6311 non-null   datetime64[ns]
 12  환불여부     318 non-null    float64       
 13  골프전환2    52 non-null     float64       
 14  양도       538 non-null    float64       
dtypes: datetime64[ns](3), float64(3), int64(8), object(1)
memory usage: 739.7+ KB


### 휴면가능 기간 처리

In [53]:
df5['휴면가능'] = np.where(df5['개월수'] >= 36, 6, np.where(df5['개월수'] >= 24, 4, np.where(df5['개월수'] >= 12, 2, 0)))

for i in range(len(df5)):
    # Convert Period to Timestamp, add DateOffset, and then convert back to Period
    df5.loc[i, '종료시점'] = (df5.loc[i, '연월'] + DateOffset(months=df5.loc[i, '개월수']))
    df5.loc[i, '휴면반영'] = (df5.loc[i, '연월'] + DateOffset(months=df5.loc[i, '개월수'] + df5.loc[i, '휴면가능']))

df6 = df5.copy()

In [54]:
# 회원카드
회원카드 = df6['회원카드번호'].unique()
행병합 = []

for i in 회원카드:
    a = df6.query('회원카드번호 == @i').sort_values(by="연월")  # 연월에 대해 데이터프레임을 정렬
    a.reset_index(drop=True, inplace=True)
    
    for j in range(len(a)) :
        a.loc[j, '거래순서'] = j  # 거래순서 열 추가 
        a.loc[j, "마지막거래"] = len(a)-1
#        개월수 = 0
        
#        for k in range (j+1) :            
#            개월수 += a.loc[k, '개월수']
        
        if j == 0:
            a.loc[j, '잔여개월수'] = a.loc[j, "개월수"]
            a.loc[j, '종료시점'] = a.loc[j,"연월"] + DateOffset(months = a.loc[j, "개월수"])
            a.loc[j, "결제시점"] = 0
            
        if (j > 0) :
            a.loc[j, '종료시점'] =  a.loc[j-1,"연월"] + DateOffset(months = a.loc[j, "개월수"])
            a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
            a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
            
            if to_datetime(a.loc[j,"연월"]) <= to_datetime('2020-12-01'):  # 코로나 이전    
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j, "결제시점"]))
            
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1, '연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j, '연월'])
                
                if a.loc[j-1, "종료시점"] >= a.loc[j, "연월"]:
                    a.loc[j, '종료시점'] =  a.loc[j-1,"종료시점"] + DateOffset(months = a.loc[j, "개월수"])
                    a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
                    a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
        
                if a.loc[j-1, "종료시점"] < a.loc[j, "연월"]: 
                    a.loc[j-1, "마지막거래"] = j-1
                    a.loc[j, "종료시점"] =   a.loc[j,"연월"] + DateOffset(months = a.loc[j, "개월수"])
                    a.loc[j, '잔여개월수'] = a.loc[j, "개월수"]
                    a.loc[j, "결제시점"] = 0
                
            elif to_datetime(a.loc[j,"연월"]) <= to_datetime('2021-01-01'):  # 코로나 기간
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j-1, "결제시점"]))
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                # 휴면가능한 경우 휴면가능 수를 추가하여 잔여 개월수를 다시 계산
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
                
                if a.loc[j-1, "종료시점"] >= a.loc[j, "연월"]:      # 이전거래와 지금 거래가 이어지는 경우
                    코로나보정값 = a.loc[j, "개월수"]+1
                    a.loc[j, '종료시점'] =  a.loc[j-1,"종료시점"] + DateOffset(months =코로나보정값) # 코로나 보정값
                    a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
                 
        
                if a.loc[j-1, "종료시점"] < a.loc[j, "연월"]:  # 지금의 거래가 이탈 이후 첫 거래인 경우
                    a.loc[j-1, "마지막거래"] = j-1
                    코로나보정값 = a.loc[j, "개월수"]+1
                    a.loc[j, "종료시점"] =   a.loc[j,"연월"] + DateOffset(months = 코로나보정값)                                        # 코로나 보정값
                    a.loc[j, '잔여개월수'] = a.loc[j, "개월수"] +1                                                                                                     # 코로나 보정값       
                    
                                    
            elif to_datetime(a.loc[j,"연월"]) <= to_datetime('2023-01-01'):  # 코로나와 리모델링 사이
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j, "결제시점"]))
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                # 휴면가능한 경우 휴면가능 수를 추가하여 잔여 개월수를 다시 계산
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
                
                if a.loc[j-1, "종료시점"] >= a.loc[j, "연월"]:
                    a.loc[j, '종료시점'] =  a.loc[j-1,"종료시점"] + DateOffset(months = a.loc[j, "개월수"]) 
                    a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
        
                if a.loc[j-1, "종료시점"] < a.loc[j, "연월"]: 
                    a.loc[j-1, "마지막거래"] = j-1
                    a.loc[j, "종료시점"] =   a.loc[j,"연월"] + DateOffset(months = a.loc[j, "개월수"])
                    a.loc[j, '잔여개월수'] = a.loc[j, "개월수"]

            elif to_datetime(a.loc[j,"연월"]) <= to_datetime('2023-02-01'):  # 리모델링 기간
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j-1, "결제시점"]))
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                # 휴면가능한 경우 휴면가능 수를 추가하여 잔여 개월수를 다시 계산
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
                
                if a.loc[j-1, "종료시점"] >= a.loc[j, "연월"]: # 이전거래와 지금 거래가 이어지는 경우
                    리모델링보정값 = a.loc[j, "개월수"]+2
                    a.loc[j, '종료시점'] =  a.loc[j-1,"종료시점"] + DateOffset(months = 리모델링보정값) # 리모델링 기간 반영
                    a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
    
        
                if a.loc[j-1, "종료시점"] < a.loc[j, "연월"]:   # 이탈 이후 첫 거래인 경우
                    a.loc[j-1, "마지막거래"] = j-1
                    리모델링보정값 = a.loc[j, "개월수"]+2
                    a.loc[j, "종료시점"] =   a.loc[j,"연월"] + DateOffset(months = 리모델링보정값)                                          # 리모델링 기간 반영
                    a.loc[j, '잔여개월수'] = a.loc[j, "개월수"] +2                                                                                                        # 리모델링 기간 반영
     
            else:  # 리모델링 이후
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j, "결제시점"]))
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
                
                if a.loc[j-1, "종료시점"] >= a.loc[j, "연월"]:
                    a.loc[j, '종료시점'] =  a.loc[j-1,"종료시점"] + DateOffset(months =a.loc[j, "개월수"])
                    a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
              
        
                if a.loc[j-1, "종료시점"] < a.loc[j, "연월"]: 
                    a.loc[j-1, "마지막거래"] = j-1
                    a.loc[j, "종료시점"] =   a.loc[j,"연월"] + DateOffset(months = a.loc[j, "개월수"])
                    a.loc[j, '잔여개월수'] = a.loc[j, "개월수"]
       
    

        if j ==0:
            if (a.loc[j, "거래순서"] == a.loc[j, "마지막거래"])&(a.loc[j,'종료시점'] < to_datetime("2024-01", format="%Y-%m")):
                a.loc[j, "이탈여부"] = 1
            if (a.loc[j, "거래순서"] == a.loc[j, "마지막거래"])&(a.loc[j,'종료시점'] >= to_datetime("2024-01", format="%Y-%m")):
                a.loc[j, "이탈여부"] = 2
                
        if j > 0:
            if a.loc[j, '결제시점'] < 0 :
                a.loc[j-1, "이탈여부"] = 1
            else: 
                a.loc[j-1, "이탈여부"] = 0
        
            if (a.loc[j, "거래순서"] == a.loc[j, "마지막거래"])&(a.loc[j,'종료시점'] < to_datetime("2024-01", format="%Y-%m")):
                a.loc[j, "이탈여부"] = 1
            if (a.loc[j, "거래순서"] == a.loc[j, "마지막거래"])&(a.loc[j,'종료시점'] >= to_datetime("2024-01", format="%Y-%m")):
                a.loc[j, "이탈여부"] = 2
           
            # a.loc[j, '잔여개월수'] = a.loc[j,"종료시점"] - a.loc[j, "연월"]
    행병합.append(a)  # 행병합 리스트에 데이터프레임 추가

# 모든 데이터프레임을 행 방향으로 병합
df7 = concat(행병합, ignore_index=True)
df7

,회원카드번호,연월,등록명,매출액,골프전환,골프매출,헬스매출,환불사용금매출,미수액,개월수,...,환불여부,골프전환2,양도,휴면가능,휴면반영,거래순서,마지막거래,잔여개월수,결제시점,이탈여부
0,12131325,2020-09-01,헬스,899000,0,0,899000,0,0,12,...,NaN,NaN,NaN,2,2021-11-01,0.0,0.0,12.0,0.0,1.0
1,13110008,2021-05-01,골프,1700000,0,1580000,0,0,0,12,...,NaN,NaN,NaN,2,NaT,0.0,2.0,13.0,0.0,0.0
2,13110008,2022-06-01,골프,1700000,0,1580000,0,0,0,12,...,NaN,NaN,NaN,2,NaT,1.0,1.0,14.0,0.0,1.0
3,13110008,2023-11-01,골프,2120000,0,1880000,0,0,0,12,...,NaN,NaN,NaN,2,2025-01-01,2.0,2.0,12.0,-3.0,2.0
4,13110009,2020-10-01,헬스,1140000,0,0,1080000,0,0,12,...,NaN,NaN,NaN,2,NaT,0.0,0.0,14.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,23120075,2023-12-01,골프,1920000,0,1690000,0,0,0,10,...,NaN,NaN,NaN,0,2024-10-01,0.0,0.0,10.0,0.0,2.0
6305,23120076,2023-12-01,헬스,990000,0,0,990000,0,0,10,...,NaN,NaN,NaN,0,2024-10-01,0.0,0.0,10.0,0.0,2.0
6306,23120077,2023-12-01,헬스,990000,0,0,990000,0,0,10,...,NaN,NaN,NaN,0,2024-10-01,0.0,0.0,10.0,0.0,2.0
6307,24010001,2023-12-01,헬스,990000,0,0,990000,0,0,10,...,NaN,NaN,NaN,0,2024-10-01,0.0,0.0,10.0,0.0,2.0


In [55]:
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6309 entries, 0 to 6308
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   회원카드번호   6309 non-null   int64         
 1   연월       6309 non-null   datetime64[ns]
 2   등록명      6309 non-null   object        
 3   매출액      6309 non-null   int64         
 4   골프전환     6309 non-null   int64         
 5   골프매출     6309 non-null   int64         
 6   헬스매출     6309 non-null   int64         
 7   환불사용금매출  6309 non-null   int64         
 8   미수액      6309 non-null   int64         
 9   개월수      6309 non-null   int64         
 10  종료시점     6309 non-null   datetime64[ns]
 11  최초등록월    6309 non-null   datetime64[ns]
 12  환불여부     318 non-null    float64       
 13  골프전환2    52 non-null     float64       
 14  양도       538 non-null    float64       
 15  휴면가능     6309 non-null   int32         
 16  휴면반영     5505 non-null   datetime64[ns]
 17  거래순서     6309 non-null   float64 

In [56]:
df7[df7['회원카드번호']==13110069]['종료시점']

31   2021-07-01
32   2022-07-01
33   2022-07-01
34   2023-09-01
35   2024-09-01
Name: 종료시점, dtype: datetime64[ns]

In [57]:
df8 =my_read_excel("변수통합.xlsx")
df8['연월'] = to_datetime(df8['연월'], format="%Y-%m")
df9 = df8.filter(['회원카드번호','연월','광고','추세값'])
df9


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500 entries, 0 to 6499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회원카드번호  6500 non-null   int64  
 1   연월      6500 non-null   object 
 2   등록명     6500 non-null   object 
 3   매출액     6500 non-null   int64  
 4   미수액     6500 non-null   int64  
 5   종료시점    6500 non-null   object 
 6   잔여개월수   6440 non-null   float64
 7   결제시점    6372 non-null   float64
 8   이탈여부    6500 non-null   int64  
 9   광고      6500 non-null   int64  
 10  추세값     6500 non-null   float64
dtypes: float64(3), int64(5), object(3)
memory usage: 558.7+ KB
None

상위 5개 행
+----+----------------+---------+----------+----------+----------+------------+--------------+------------+------------+--------+-----------+
|    |   회원카드번호 | 연월    | 등록명   |   매출액 |   미수액 | 종료시점   |   잔여개월수 |   결제시점 |   이탈여부 |   광고 |    추세값 |
|----+----------------+---------+----------+----------+----------+------------+----------

,회원카드번호,연월,광고,추세값
0,12131325,2020-09-01,0,-0.131868
1,13110008,2021-05-01,0,-0.263736
2,13110008,2022-06-01,1,-0.285714
3,13110008,2023-11-01,0,3.500000
4,13110009,2020-10-01,0,0.840659
...,...,...,...,...
6495,23120076,2023-12-01,0,0.000000
6496,23120077,2023-12-01,0,0.000000
6497,24010001,2023-12-01,0,0.000000
6498,24010003,2022-01-01,1,0.000000


In [59]:
merged_df = merge(df7, df9, on=['회원카드번호','연월'], how="left")
merged_df.to_excel('통합6-1.xlsx', index=False)